In [2]:
%cd ..
# %pip install .

%load_ext autoreload

%autoreload 2

/home/christos/Repos/neuropixel-pipeline


In [3]:
import datajoint as dj

dj.config["database.host"] = "at-database.ad.bcm.edu"

from neuropixel_pipeline.readers.labview import LabviewNeuropixelMeta
from neuropixel_pipeline.schemata import probe, ephys

from neuropixel_pipeline.utils import TODO

from pathlib import Path

Connecting cpapadop@at-database.ad.bcm.edu:3306


## Probe

In [3]:
probe.ProbeType.fill_neuropixel_probes()
probe.ProbeType()

probe_type e.g. neuropixels_1.0
neuropixels 1.0 - 3A
neuropixels 1.0 - 3B
neuropixels 2.0 - MS
neuropixels 2.0 - SS
neuropixels UHD


In [41]:
probe.Probe.heading

# Represent a physical probe with unique identification
probe                : varchar(32)                  # unique identifier for this model of probe (e.g. serial number)
---
probe_type           : varchar(32)                  # e.g. neuropixels_1.0
probe_comment=""     : varchar(1000)                # 

In [4]:
probe.Probe.insert1(
    dict(
        probe="19398116102",
        probe_type="neuropixels 1.0 - 3B",
        probe_comment="Used in maria's recordings",
    ),
    skip_duplicates=True,
)
probe.Probe()

probe unique identifier for this model of probe (e.g. serial number),probe_type e.g. neuropixels_1.0,probe_comment
19398116102,neuropixels 1.0 - 3B,Used in maria's recordings


# PreClustering

In [5]:
# Need to get from this to accessible kilosort dir
key = {"animal_id": 29187, "session": 25, "scan_idx": 1}

experiment = dj.create_virtual_module("experiment", "pipeline_experiment")

session_rel = experiment.Session() & key
session_rel.fetch1()

{'animal_id': 29187,
 'session': 25,
 'rig': 'RS1',
 'session_date': datetime.date(2023, 5, 16),
 'username': 'maria',
 'anesthesia': 'awake',
 'scan_path': 'W:\\Two-Photon\\maria\\2023-05-16_13-39-49',
 'behavior_path': 'W:\\Two-Photon\\maria\\2023-05-16_13-39-49',
 'craniotomy_notes': ' ',
 'session_notes': ' ',
 'archive': 'RS1A2A',
 'session_ts': datetime.datetime(2023, 5, 16, 13, 39, 49)}

In [6]:
ephys.Session.add_session(key)
ephys.Session()

session_id Session primary key hash,animal_id animal id,session original session id,scan_idx scan idx
1,29187,25,1


In [45]:
# Uses contents only, lookup table
ephys.AcquisitionSoftware()

acq_software
LabviewV1
Open Ephys
SpikeGLX


In [8]:
probe.Probe()

probe unique identifier for this model of probe (e.g. serial number),probe_type e.g. neuropixels_1.0,probe_comment
19398116102,neuropixels 1.0 - 3B,Used in maria's recordings


In [9]:
session_dir = Path("/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16")

labview_metadata = LabviewNeuropixelMeta.from_h5(session_dir)

insertion_key = dict(session_id=1, insertion_number=0)

ephys.ProbeInsertion.insert1(
    dict(
        **insertion_key,
        probe=labview_metadata.serial_number,
    ),
    skip_duplicates=True,
)
ephys.ProbeInsertion()

session_id Session primary key hash,insertion_number,probe unique identifier for this model of probe (e.g. serial number)
1,0,19398116102


In [10]:
# manual_insert_data = TODO('manual insertion data')
# ephys.InsertionLocation.insert(dict())
ephys.InsertionLocation()

session_id Session primary key hash,insertion_number,skull_reference,ap_location (um) anterior-posterior; ref is 0; more anterior is more positive,ml_location (um) medial axis; ref is 0 ; more right is more positive,depth (um) manipulator depth relative to surface of the brain (0); more ventral is more negative,"theta (deg) - elevation - rotation about the ml-axis [0, 180] - w.r.t the z+ axis","phi (deg) - azimuth - rotation about the dv-axis [0, 360] - w.r.t the x+ axis","beta (deg) rotation about the shank of the probe [-180, 180] - clockwise is increasing in degree - 0 is the probe-front facing anterior"


In [11]:
ephys.EphysFile.insert1(
    dict(
        file_path=session_dir,
        acq_software="LabviewV1",
    ),
    skip_duplicates=True,
)
ephys.EphysRecording.populate()
ephys.EphysRecording()

session_id Session primary key hash,insertion_number,file_path file path or directory for an ephys session,electrode_config_hash,sampling_rate (Hz),recording_datetime datetime of the recording from this probe,recording_duration (seconds) duration of the recording from this probe
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,34c99413-1867-1851-a506-633f7ad51902,30000.0,None,nan


In [12]:
probe.ElectrodeConfig()

electrode_config_hash,probe_type e.g. neuropixels_1.0,electrode_config_name user friendly name
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,


In [13]:
probe.ElectrodeConfig.Electrode()

electrode_config_hash,probe_type e.g. neuropixels_1.0,"electrode electrode index, starts at 0"
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,0
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,1
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,2
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,3
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,4
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,5
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,6
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,7
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,8
34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,9


In [14]:
# Mostly fully computed (from neuropixel data I think)
# ephys.LFP.populate()

# Clustering

In [15]:
# lookup table
ephys.ClusteringMethod()

clustering_method,clustering_method_desc
kilosort2,kilosort2 clustering method
kilosort2.5,kilosort2.5 clustering method
kilosort3,kilosort3 clustering method
kilosort4,kilosort4 clustering method


In [16]:
settings = {
    "NchanTOT": 385,
    "fs": 30000,
    "nt": 61,
    "Th": 8,
    "spkTh": 8,
    "Th_detect": 9,
    "nwaves": 6,
    "nskip": 25,
    "nblocks": 5,
    "binning_depth": 5,
    "sig_interp": 20,
    "probe_name": "neuropixPhase3B1_kilosortChanMap.mat",
}
settings["nt0min"] = int(20 * settings["nt"] / 61)
settings["NT"] = 2 * settings["fs"]
settings["n_chan_bin"] = settings["NchanTOT"]


# # lookup table, more commonly used (need a simple interface to insert into this occasionally)
ephys.ClusteringParamSet.fill(
    params=settings,
    clustering_method="kilosort4",
    description="default kilosort4 params",
    skip_duplicates=True,
)

ephys.ClusteringParamSet().fetch1()

{'paramset_idx': 1,
 'clustering_method': 'kilosort4',
 'paramset_desc': 'default kilosort4 params',
 'paramset_hash': UUID('0ac0bdfb-1a29-525f-ec6a-ea65229830cd'),
 'params': {'NchanTOT': 385,
  'fs': 30000,
  'nt': 61,
  'Th': 8,
  'spkTh': 8,
  'Th_detect': 9,
  'nwaves': 6,
  'nskip': 25,
  'nblocks': 5,
  'binning_depth': 5,
  'sig_interp': 20,
  'probe_name': 'neuropixPhase3B1_kilosortChanMap.mat',
  'nt0min': 20,
  'NT': 60000,
  'n_chan_bin': 385}}

In [17]:
# lookup table
ephys.ClusterQualityLabel()

"cluster_quality_label cluster quality type - e.g. 'good', 'MUA', 'noise', etc.",cluster_quality_description
good,single unit
mua,multi-unit activity
noise,bad unit
ok,"probably a single unit, but could be contaminated"


In [18]:
task_source_rel = (ephys.EphysRecording & insertion_key).proj() * (
    ephys.ClusteringParamSet() & {"paramset_idx": 1}
).proj()
task_source_key = task_source_rel.fetch1()

task_source_key["clustering_output_dir"] = session_dir / "spikes_kilosort4"
task_source_key["task_mode"] = "load"
ephys.ClusteringTask.insert1(task_source_key, skip_duplicates=True)
ephys.ClusteringTask()

session_id Session primary key hash,insertion_number,file_path file path or directory for an ephys session,paramset_idx,clustering_output_dir clustering output directory relative to the clustering root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation"
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16/spikes_kilosort4,load


In [19]:
ephys.Clustering.populate()
ephys.Clustering()

session_id Session primary key hash,insertion_number,file_path file path or directory for an ephys session,paramset_idx,clustering_time time of generation of this set of clustering results,package_version
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,2023-05-25 14:33:30,


In [20]:
# lookup table
ephys.CurationType()

curation
no curation


In [21]:
from datetime import datetime

curation_source_key = (ephys.Clustering() & task_source_key).proj().fetch1()
# print(curation_source_key)

ephys.Curation.insert1(
    dict(
        **curation_source_key,
        curation_id=0,
        curation_time=datetime.now(),  # might not always be true
        curation_output_dir=curation_source_key["file_path"],
        quality_control=False,
        curation="no curation",
        curation_note="notes on the curation",
    ),
    skip_duplicates=True,
)

ephys.Curation()

session_id Session primary key hash,insertion_number,file_path file path or directory for an ephys session,paramset_idx,curation_id,curation_time time of generation of this set of curated clustering results,"curation_output_dir output directory of the curated results, relative to root data directory",quality_control has this clustering result undergone quality control?,curation,curation_note
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,2023-05-25 14:33:30,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,0,no curation,notes on the curation


In [22]:
ephys.CuratedClustering.populate()
ephys.CuratedClustering()

session_id Session primary key hash,insertion_number,file_path file path or directory for an ephys session,paramset_idx,curation_id
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0


In [28]:
ephys.CuratedClustering.Unit()

session_id Session primary key hash,insertion_number,file_path file path or directory for an ephys session,paramset_idx,curation_id,unit_id,electrode_config_hash,probe_type e.g. neuropixels_1.0,"electrode electrode index, starts at 0","cluster_quality_label cluster quality type - e.g. 'good', 'MUA', 'noise', etc.",spike_count how many spikes in this recording for this unit,"spike_times (s) spike times of this unit, relative to the start of the EphysRecording",spike_sites array of electrode associated with each spike,"spike_depths (um) array of depths associated with each spike, relative to the (0, 0) of the probe"
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,0,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,3,good,11649,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,1,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,3,mua,4096,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,2,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,2,mua,7744,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,3,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,1,good,2332,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,4,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,2,mua,40955,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,5,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,1,mua,233,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,6,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,2,good,13162,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,7,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,1,good,1329,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,8,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,1,good,7375,=BLOB=,=BLOB=,=BLOB=
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_14-59-51/2023-05-16_15-00-16,1,0,9,34c99413-1867-1851-a506-633f7ad51902,neuropixels 1.0 - 3B,0,mua,8533,=BLOB=,=BLOB=,=BLOB=


# PostClustering

In [ ]:
ephys.WaveformSet.populate()  # requires mean waveforms from ecephys_spike_sorting package (mean_waveforms.npy)
ephys.WaveformSet()

In [ ]:
ephys.QualityMetrics.populate()  # requires quality metrics from ecephys_spike_sorting package (metrics.csv)
ephys.QualityMetrics()

In [5]:
ephys.ClusteringTask.proj()

session_id Session primary key hash,insertion_number,file_path file path or directory for an ephys session,paramset_idx
1,0,/mnt/scratch09/Ephys/Mouse/2023-05-16_10-22-55/2023-05-16_10-27-01,1
